# Libraries



In [ ]:
!pip install torch diffusers  transformers pillow matplotlib mediapipe opencv-python opencv-contrib-python gradio  torchvision requests beautifulsoup4 fake_useragent


In [ ]:
!apt-get update
!apt-get install -y wget unzip xvfb libxi6 libgconf-2-4
!apt-get install -y libappindicator1 fonts-liberation
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install
!rm google-chrome-stable_current_amd64.deb

!pip install selenium requests webdriver-manager



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,221 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,046 B in 1s (2,136 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repos

In [ ]:
%%capture

!pip install groq langchain_community sentence_transformers
!pip install llama-index-llms-groq
!pip install groq




# Data Scrapping

In [ ]:
import os
import logging
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import shutil
import urllib.parse

# Configure logging
logging.basicConfig(level=logging.INFO)

# Websites to scrape
websites = {
    'junaidjamshed': 'https://www.junaidjamshed.com/womens/kurti.html?product_list_dir=desc&product_list_order=top_rated',
    'khaadi': 'https://pk.khaadi.com/ready-to-wear/essentials/kurta/kurta/?prefn1=filter_categories&prefv1=Kurta&srule=most-popular&start=0&sz=96',
}

# Keywords to filter images (specific to shirts)
keywords = ['shirt', 'kurta', 'kurti']

# Folder to save images
output_folder = "scraped_images"

# Clear output folder before scraping
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # Delete the folder and its contents
os.makedirs(output_folder, exist_ok=True)  # Recreate the folder

# Selenium setup
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.binary_location = "/usr/bin/google-chrome"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Function to scrape images from Junaid Jamshed using Selenium
def scrape_images_junaidjamshed(site_name, url):
    try:
        driver.get(url)

        # Scroll to load all images
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down incrementally and wait for images to load
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Wait until images are loaded (adjust based on your page structure)
        time.sleep(3)

        images = []
        img_elements = driver.find_elements(By.TAG_NAME, "img")

        for img in img_elements:
            img_url = img.get_attribute('src') or img.get_attribute('data-src') or img.get_attribute('srcset')
            alt_text = img.get_attribute('alt')

            # Skip base64 images
            if img_url and img_url.startswith('data:image'):
                continue

            # Filter images by keywords in alt text
            if alt_text and any(keyword.lower() in alt_text.lower() for keyword in keywords):
                images.append((img_url, alt_text))

        # Save top 10 images
        save_images(site_name, images)

    except Exception as e:
        logging.error(f"Error scraping {site_name}: {e}")

# Function to scrape images from Khaadi using BeautifulSoup
def scrape_images_khaadi(site_name, url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        images = []
        img_elements = soup.find_all('img')

        seen_urls = set()
        for img in img_elements:
            img_url = img.get('src') or img.get('data-src')
            alt_text = img.get('alt')

            # Skip base64 images
            if img_url and img_url.startswith('data:image'):
                continue

            # Handle relative URLs
            img_url = urllib.parse.urljoin(url, img_url)

            # Skip duplicate URLs
            if img_url in seen_urls:
                continue
            seen_urls.add(img_url)

            # Filter images by keywords in alt text
            if alt_text and any(keyword.lower() in alt_text.lower() for keyword in keywords):
                images.append((img_url, alt_text))

        # Save top 10 images
        save_images(site_name, images)

    except Exception as e:
        logging.error(f"Error scraping {site_name}: {e}")

# Function to save images and descriptions
def save_images(site_name, images):
    for i, (img_url, description) in enumerate(images[:10]):
        try:
            if img_url and img_url.startswith('http'):
                img_data = requests.get(img_url, timeout=10).content
                if img_data:
                    img_name = f"{site_name}shirt{i + 1}.jpg"
                    img_path = os.path.join(output_folder, img_name)

                    # Save image
                    with open(img_path, 'wb') as img_file:
                        img_file.write(img_data)

                    # Save description
                    desc_path = os.path.join(output_folder, f"{site_name}shirt{i + 1}.txt")
                    with open(desc_path, 'w') as desc_file:
                        desc_file.write(description)

                    logging.info(f"Saved {img_name} with description: {description}")
                else:
                    logging.error(f"Image {img_url} is empty, skipping...")
            else:
                logging.warning(f"Invalid image URL: {img_url}")
        except Exception as e:
            logging.error(f"Failed to save image {img_url}: {e}")

# Load images from the output folder
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            images.append(filename)
    return images

# Main scraping function
def scrape_data():
    all_images = []
    for site_name, url in websites.items():
        logging.info(f"Scraping {site_name} for shirts...")
        if site_name == 'junaidjamshed':
            scrape_images_junaidjamshed(site_name, url)
        elif site_name == 'khaadi':
            scrape_images_khaadi(site_name, url)

    # Reload the gallery images after scraping
    all_images = load_images_from_folder(output_folder)
    return all_images

# Start scraping
images = scrape_data()
logging.info(f"Scraping complete. Images saved: {images}")

# Close the driver
driver.quit()

# Generative Model

In [ ]:
import gradio as gr
import os
from PIL import Image
import torch
from diffusers import StableDiffusionImg2ImgPipeline
import torchvision.transforms as transforms
from typing import List, Dict
from groq import Groq

# Set up Groq API key
os.environ["GROQ_API_KEY"] = "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"
client = Groq()  # Initialize Groq API client
DEFAULT_MODEL = "llama-3.1-70b-versatile"

# Function to create assistant message format
def assistant(content: str):
    return {"role": "assistant", "content": content}

# Function to create user message format
def user(content: str):
    return {"role": "user", "content": content}

# Function for chat completion with Groq
def chat_completion(messages: List[Dict], model=DEFAULT_MODEL, temperature=0.6, top_p=0.9) -> str:
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature,
        top_p=top_p,
    )
    return response.choices[0].message.content

# Preprocess input images
def preprocess_image(image_path, size=(512, 512)):
    image = Image.open(image_path).convert("RGB")
    resize_transform = transforms.Resize(size)
    return resize_transform(image)

# Load all images from the 'scraped_images' folder
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            image = preprocess_image(image_path)
            images.append((image, image_path))  # Store image and path tuple
    return images

# Load all text descriptions from the 'scraped_images' folder
def load_descriptions_from_folder(folder_path):
    descriptions = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                description = file.read().strip()
                descriptions[filename.replace(".txt", ".jpg")] = description
    return descriptions

# Path to the folder where the scraped images are saved
scraped_images_folder = "scraped_images"

# Global variables to hold loaded images and descriptions
images = []
descriptions = {}

# Function to save uploaded image and description without re-fetching all data
def save_uploaded_data(uploaded_image, description):
    global images, descriptions

    if uploaded_image is not None:
        # Save the image to the 'scraped_images' folder
        image_path = os.path.join(scraped_images_folder, os.path.basename(uploaded_image))
        try:
            os.rename(uploaded_image, image_path)
        except Exception as e:
            print(f"Error renaming file: {e}")
            return gr.update()

        # Save the description if provided
        if description:
            description_path = os.path.join(scraped_images_folder, os.path.basename(uploaded_image).replace(".jpg", ".txt"))
            with open(description_path, "w", encoding="utf-8") as file:
                file.write(description)

        # Preprocess and add the new image to the existing images list (without reloading all images)
        new_image = preprocess_image(image_path)
        images.append((new_image, image_path))

        # Add the new description to the existing descriptions dictionary
        descriptions[os.path.basename(image_path)] = description

        # Update the gallery with the new image only
        return gr.update(value=[img[0] for img in images])  # Only update the gallery with the current images
    return gr.update()

# Load the Stable Diffusion pipeline
def load_pipeline():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    return StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16
    ).to(device)

# Sync sliders to ensure they sum to 1
def sync_sliders(value):
    """ Ensure alpha1 and alpha2 sliders always sum to 1 """
    if value < 0:
        value = 0
    elif value > 1:
        value = 1
    return 1 - value

# Blend and generate image using the Stable Diffusion pipeline
def blend_and_generate_image(image_paths, alpha1, alpha2, generated_prompt):
    if not image_paths or len(image_paths) < 1:
        return None

    # Ensure alpha1 and alpha2 are within valid range and sum to 1
    alpha1 = max(0, min(1, alpha1))
    alpha2 = max(0, min(1, alpha2))
    if alpha1 + alpha2 != 1:
        alpha2 = 1 - alpha1  # Adjust alpha2 to maintain the sum to 1

    # Open images and preprocess them
    image1 = Image.open(image_paths[0]).convert("RGBA")

    # If we only have one image, we'll use it directly (no blending required)
    blended_image = image1

    # Generate a new image using the blended image and the prompt
    pipe = load_pipeline()
    output_image = pipe(
        prompt=generated_prompt,
        image=blended_image.convert("RGB"),
        strength=0.80,  # Influence of the input image
        guidance_scale=7.5,  # Balances adherence to the prompt
        num_inference_steps=50,
        generator=torch.manual_seed(42),  # Fixed seed for reproducibility
    ).images[0]

    return output_image

# Generate a creative prompt based on selected images' descriptions
def generate_prompt_from_selected_images(image_paths):
    selected_descriptions = []

    # Get descriptions for the selected images
    for image_path in image_paths:
        image_name = os.path.basename(image_path)
        description = descriptions.get(image_name, "")
        if description:
            selected_descriptions.append(description)

    # Generate creative input for the prompt using chat completion (Groq prompt)
    if selected_descriptions:
        groq_prompt = chat_completion([user(f'generate a new kurta design prompt based on these descriptions: {", ".join(selected_descriptions)}')])
        return groq_prompt
    else:
        return "No descriptions found for the selected images."

# Define the Gradio app
def create_ui():
    with gr.Blocks() as app:
        # Title/Heading Section
        with gr.Row():
            gr.Markdown("### Select images and generate a new image based on descriptions and blending")

        # Buttons and Inputs Section (Upload and Description)
        with gr.Row():
            fetch_data_button = gr.Button("Fetch New Data")
            upload_image = gr.File(label="Upload Image", type="filepath", file_types=[".jpg"])
            description_input = gr.Textbox(label="Image Description")
            upload_button = gr.Button("Upload")

        # Image Gallery Section
        with gr.Row():
            gallery = gr.Gallery(
                label="Loaded Images",
                value=[],  # Placeholder for images (you may update this dynamically later)
                interactive=True,
                columns=4,
                height="auto",
            )

        # Display selected image paths
        with gr.Row():
            image1_display = gr.Textbox(label="First Selected Image Path", interactive=False)
            image2_display = gr.Textbox(label="Second Selected Image Path", interactive=False)

        # Display the generated prompt
        with gr.Row():
            generated_prompt_display = gr.Textbox(label="Generated Prompt", interactive=True, lines=3)

        # Alpha sliders for blending
        with gr.Row():
            alpha1_slider = gr.Slider(0, 1, value=0.5, step=0.01, label="Weight of First Image")
            alpha2_slider = gr.Slider(0, 1, value=0.5, step=0.01, label="Weight of Second Image")

        # Output for generated image
        with gr.Row():
            output_generated_image = gr.Image(label="Generated Image", type="pil", interactive=False)

        # Handle image selection
        selected_images = []

        def handle_selection(evt: gr.SelectData):
            selected_path = images[evt.index][1]  # Get the path (second element of the tuple)
            if len(selected_images) < 2:
                selected_images.append(selected_path)
            else:
                selected_images.pop(0)
                selected_images.append(selected_path)

            # Generate and update the prompt based on selected images
            if len(selected_images) > 0:
                generated_prompt = generate_prompt_from_selected_images(selected_images)
                return selected_images[0] if len(selected_images) > 0 else "", selected_images[1] if len(selected_images) > 1 else "", generated_prompt
            return "", "", ""

        gallery.select(handle_selection, None, [image1_display, image2_display, generated_prompt_display])

        # Update sliders dynamically and ensure they sum to 1
        alpha1_slider.change(sync_sliders, inputs=alpha1_slider, outputs=alpha2_slider)
        alpha2_slider.change(sync_sliders, inputs=alpha2_slider, outputs=alpha1_slider)

        # Fetch new data
        def fetch_new_data_and_update_gallery():
            global images, descriptions
            updated_images = load_images_from_folder(scraped_images_folder)
            descriptions = load_descriptions_from_folder(scraped_images_folder)  # Ensure descriptions are loaded
            images = updated_images  # Update the global images list
            return gr.update(value=[img[0] for img in updated_images])

        fetch_data_button.click(fetch_new_data_and_update_gallery, None, gallery)

        # Handle uploading image and description
        upload_button.click(save_uploaded_data, [upload_image, description_input], gallery)

        # Generate blended image
        def blend_and_generate(image1_path, image2_path, alpha1, alpha2, generated_prompt):
            return blend_and_generate_image(
                [image1_path, image2_path],  # Pass image paths instead of tuples
                alpha1,
                alpha2,
                generated_prompt
            )

        generate_button = gr.Button("Generate Image")
        generate_button.click(
            blend_and_generate,
            [image1_display, image2_display, alpha1_slider, alpha2_slider, generated_prompt_display],  # Include the prompt here
            output_generated_image
        )

    return app

# Launch the Gradio app
if __name__ == "__main__":
    app = create_ui()
    app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa4475837d61ab6b52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
